* 언어모델의 성능 측정을 위한 방법 중 하나, 특히 번역성능 측정의 대표적인 방법임

In [1]:
import numpy as np
from collections import Counter
from nltk import ngrams

### 1. BLEU

* BLEU는 기계번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법임
* 측정 기준은 n-gram에 기반하며, PPL보다 성능이 좋음

-----
* BLEU를 이해하기 위해 기계번역 성능평가를 위한 몇가지 직관적인 방법을 제시하고, 문제점을 보완한 방식 순으로 설명


#### 1-1. 단어 개수 카운트로 측정하기 (Unigram Precision)


* 한국어-영어 번역기의 성능을 측정한다고 가정하면, 두개의 기계번역기가 존재하고 두 기계 번역기에 같은 한국어 문장을 입력하여 번역된 영어 문장의 성능을 측정함
* 번역된 문장을 각각 candidate 1, candidate 2라고 하면, 이 문장의 성능을 평가하기 위해 정답으로 비교되는 문장이 있어야 함
* 세명의 사람에게 한국어를 보고 영작해보라고 하여 세개의 번역문장을 만들어냄. 이 세문장을 각각 Reference 1, 2,3이라고 하자

**Example 1**

* Ca 1 : It is a guide to action which ensures that the military always obeys the commands of the party.
* Ca 2 : It is to insure the troops forever hearing the activity guidebook that party direct.
* Ref 1 : It is a guide to action that ensures that the military will forever heed Party commands.
* Ref 2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.
* Ref 3 : It is the practical guide for the army always to heed the directions of the party.

* 가장 직관적인 성능평가방법은 Ref 1, 2, 3 중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것임
* 그 후에 Ca의 모든 단어의 카운트 합. 즉, Ca의 총 단어 수로 나눠 주는데, 이 방법을 **유니그램 정밀도(Unigram Precision)**라고 함

$\text{Unigram Precision =}\frac{\text{Ref들 중에서 존재하는 Ca의 단어의 수}}{\text{Ca의 총 단어 수}} = \frac{\text{the number of Ca words(unigrams) which occur in any Ref}}{\text{the total number of words in the Ca}}$

위의 계산 방법에 따르면 Ca1과 Ca2의 유니그램 정밀도는 다음 같음

$$\text{Ca1 Unigram Precision =} \frac{17}{18}$$

$$\text{Ca2 Unigram Precision =} \frac{8}{14}$$

#### 1-2. 중복을 제거하여 보정하기 (Modified Unigram Precision)

**Example 2**

* Candidate : the the the the the the the
* Reference1 : the cat is on the mat
* Reference2 : there is a cat on the mat

----
위의 Ca는 the만 7개가 등장한 안좋은 번역이지만, 위의 유니그램으로 평가하면 최고의 성능평가(7/7=1)를 받게 됨. 따라서 유니그램 정밀도를 보정할 필요가 있음

이를 보정하기 위해 정밀도의 분자를 계산하기 위해 Ref와 매칭하며 카운트하는 과정에서 Ca의 유니그램이 이미 Ref에서 매칭된 적이 있었는지를 고려해서 평가해야 함

$$\text{Unigram Precision =}\frac{\text{Ref들과 Ca를 고려한 새로운 카운트 방법이 필요!}}{\text{Ca의 총 유니그램 수}}$$

정밀도의 분자를 계산하기 위한 각 유니그램의 카운트는 다음과 같이 수정함

* 유니그램이 하나의 Ref에서 최대 몇번 등장하는지 카운트 ==> Max_Ref_Count
    $$Count_{clip}\ =\ min(Count,\ Max$$
    
* 위의 카운트를 사용하여 분자를 계산한 정밀도를 보정된 유니그램 정밀도(Modified Unigram Precision)라고 함
$$\text{Modified Unigram Precision =}\frac{\text{Ca의 각 유니그램에 대해 }Count_{clip}\text{을 수행한 값의 총 합}}{\text{Ca의 총 유니그램 수}}=\frac{\sum_{unigram∈Candidate}\ Count_{clip}(unigram)}
{\sum_{unigram∈Candidate}\ Count(unigram)}$$

* 분모는 이전과 동일하게 Ca의 모든 유니그램에 대해 각각 Count하고 모두 합한 값을 사용함
* 보정된 유니그램 정밀도를 이용하면, Example2에서는 the의 경우 Ref1에서 2번 등장하였으므로, the의 카운트는 2로 보정됨. 이렇게 되면 정밀도는 2/7로 변경됨

---
#### 1-3. 보정도니 유니그램 정밀도(Modified Unigram Precision) 구현하기
1. 분모의 유니그램을 카운트하는 Count 함수
2. 분자의 유니그램을 카운트하는 $Count_{clip}$ 함수 
3. 유니그램을 단순히 카운트하는 함수 simple_count

In [2]:
# 토큰화된 문장(tokens)에서 n-gram을 카운트 : 유니그램의 경우 n=1
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

In [3]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."

tokens = candidate.split()
result = simple_count(tokens, 1)
print('유니그램 카운트 :', result)

유니그램 카운트 : Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


* 위의 출력 결과는 모든 유니그램을 카운트한 결과를 보여줌. 대부분의 유니그램이 1개씩 카운트되었으나, 유니그램 the는 문장에서 3번 등장하였으므로 유일하게 3의 값을 가짐

In [4]:
candidate = 'the the the the the the the'
tokens = candidate.split()
result = simple_count(tokens, 1)
print('유니그램 카운트 :', result)

유니그램 카운트 : Counter({('the',): 7})


In [5]:
for n_gram, cnt in result.items():
    print(n_gram, cnt)

('the',) 7


In [63]:
## 분자에 해당하는 Count_clip함수 구현

def count_clip(candidate, reference_list, n):
    # ca문장을 n-gram 카운트
    ca_cnt = simple_count(candidate.split(), n)
    #print(ca_cnt)
    max_ref_cnt_dict = dict()
    
    for ref in reference_list:
        # ref 문장에서 n-gram 카운트
        ref_cnt = simple_count(ref.split(), n)
        
        # 각 ref 문장에 대해 비교하여 n-gram의 최대 등장횟수를 계산
        for n_gram, cnt in ref_cnt.items():
            try:
                max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
            except:
                max_ref_cnt_dict[n_gram] = cnt
                
    return {
        # count_clip = min(count, max_ref_count)
        n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram, cnt in ca_cnt.items()
    }
        
    

In [64]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]

In [65]:
count_clip(candidate, references, 1)

{('the',): 2}

동일한 예제 문장에 대해 위의 simple_count 함수는 the가 7개로 카운트되었던 것과 달리, 이번에는 2개로 카운트됨. 위의 두 함수를 사용하여 예제문장에 대해 보정된 정밀도 연산하는 함수를 modified_precision 이름의 함수를 구현함

In [72]:
def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n)
    total_clip_cnt = sum(clip_cnt.values())  # 분자
    cnt = simple_count(candidate.split(), n)
    total_cnt = sum(cnt.values())  # 분모
    # 분모가 0이 되는 것을 방지
    if total_cnt == 0:
        total_cnt = 1
    # 분자 : count_clip의 합, 분모 : 단순 count의 합
    return (total_clip_cnt / total_cnt)

In [73]:
modified_precision(candidate, references, 1)

0.2857142857142857

#### 1-4. 순서를 고려하기 위해 n-gram으로 확장하기
* BoW 표현과 유사하게 유니그램 정밀도와 같이 각 단어의 빈도수로 접근하는 방법은 단어의 순서를 고려하지 않다는 것임

**Example 1**

* Candidate1 : It is a guide to action which ensures that the military always obeys the commands of the party.
* Candidate2 : It is to insure the troops forever hearing the activity guidebook that party direct.
* Candidate3 : the that military a is It guide ensures which to commands the of action obeys always party the.
* Reference1 : It is a guide to action that ensures that the military will forever heed Party commands.
* Reference2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.
* Reference3 : It is the practical guide for the army always to heed the directions of the party.

Ca3은 Ca1에서 모든 유니그램의 순서를 랜덤으로 섞은 것으로 영어문법에 맞지 않은 문장임. 하지만 Ref1, 2, 3과 비교하여 유니그램 정밀도를 적용하면 ca1, ca3의 정밀도는 동일함.

유니그램 정밀도는 유니그램의 순서를 전혀 고려하지 않았기 때문임. 따라서 개별적인 유니그램 / 단어로서 카운트하는 유니그램 정밀도에서 다음에 등장한 단어까지 함께 고려하고 Bigram, Trigram, 4-gram 단위 등으로 계산한 정밀도인 n-gram을 이용한 정밀도 측정방법 도입

이들 각각은 카운트 단위를 2개, 3개, 4개로 보느냐의 차이로 2-gram Precision, 3-gram precision, 4-gram precision이라고 하기도 함

**Example 2**

* Candidate1 : the the the the the the the
* Candidate2 : the cat the cat on the mat
* Reference1 : the cat is on the mat
* Reference2 : there is a cat on the mat

결과적으로 ca2의 바이그램 정밀도는 4/6이 됨. 반면, ca1의 바이그램 정밀도는 0임

이 보정된 정밀도를 식으로 정의하면 다음과 같음
$$p_{1}=\frac{\sum_{unigram∈Candidate}\ Count_{clip}(unigram)}
{\sum_{unigram∈Candidate}\ Count(unigram)}$$

이를 n-gram으로 일반화하면 아래와 같음

$$p_{n}=\frac{\sum_{n\text{-}gram∈Candidate}\ Count_{clip}(n\text{-}gram)}
{\sum_{n\text{-}gram∈Candidate}\ Count(n\text{-}gram)}$$

BLEU는 보정된 정밀도 $p_{1}, p_{2}, ..., p_{n}$를 모두 조합하여 사용하는데, 이를 조합한 BLEU 식은 아래와 같음

$$BLEU = exp(\sum_{n=1}^{N}w_{n}\ \text{log}\ p_{n})$$

* $w_n$: 각 gram의 보정된 정밀도에서 서로 다른 가중치를 줄수 있음. 이 가중치의 합은 1이며, 예를 들어, N이 4라고 한다면, p1, p2, p3, p4 각각에 대해 동일한 가중치를 주고자 한다면 모두 0.25를 적용할 수 있음

* 그러나 위의 BLEU 식에서도 문제점이 있음


#### 1-5. 짧은 문장 길에 대한 패널티(Brevity Penalty)

* n-gram으로 단어의 순서를 고려한다고 하더라도 문장 길에 BLEU의 점수가 과한 영향을 받을 수 있음


**Example 1**
* Candidate4 : it is

이 문장은 유니그램 정밀도나 바이그램 정밀도가 각각 2/2, 1/1로 두 정밀도 모두 1이라는 높은 정밀도를 얻음. 이와 같이 제대로 도니 번역이 아님에도 문장의 길이가 짧다는 이유로 높은 점수를 받는 것은 이상함. 그래서 문장길이가 짧은 경우에는 점수에 패널티를 줄 필요가 있음. 이를 Brevity Penalty라고 함


**Example 3**
* Candidate 1: I always invariably perpetually do.
* Candidate 2: I always do.
* Reference 1: I always do.
* Reference 2: I invariably do.
* Reference 3: I perpetually do.

Example3에서 ca1은 가장 많은 단어를 사용했지만, ca2보다 좋지 못한 번역임. 즉, Ref의 단어를 가장 많이 사용한 것이 꼭 좋은 번역이라는 의미는 아님. 따라서 Brevity Penalty를 설계할 때 이 경우는 제외해야 함

다시 Ref보다 Ca의 길이가 짧을 경우, 패널티는 앞서 배운 BLEU 식에 곱하는 방식으로 사용함. Brevity Penalty를 줄여서 BP라고 할 때, 최종 식은 다음과 같음

$$BLEU = BP × exp(\sum_{n=1}^{N}w_{n}\ \text{log}\ p_{n})$$

위의 수식은 패널티를 줄 필요가 없는 경우, BP의 값이 1이어야 함을 의미함. 따라서 BP의 수식은 다음과 같음

$$BP = \begin{cases}1&\text{if}\space c>r\\ e^{(1-r/c)}&\text{if}\space c \leq r \end{cases}$$

* c : Candidate 길이
* r : Candidate와 가장 길이 차이가 작은 Reference 길이

Ref가 1개라면 Ca와 Ref의 두 문장의 길이만을 가지고 계산하면 되겠지만, 여기서는 Ref가 여러 개일 때를 가정하므로, r은 모든 Ref 중에서 Ca와 가장 길이차이가 작은 Ref의 길이로 함


In [87]:
# Ca 길이와 가장 근접한 Ref 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate.split())
    ref_lens = [len(ref.split()) for ref in reference_list]
    # 길이 차이를 최소화하는 Ref 찾아서 그 길이를 리턴
    closest_ref_len = min(abs(ref-ca_len) for ref in ref_lens)
    return closest_ref_len

In [88]:
candidate = "I always do."
reference = ["I always do."]
closest_ref_length(candidate, reference)

0

만약 Ca와 길이가 동일한 Ref가 있다면 길이 차이가 0인 수준의 매치(best match length)임. 만약 서로 다른 길이의 Ref이지만, Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택함

예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택함. closest_ref_length 함수를 통해 r를 구하면, BP를 구하는 함수 brevity_penalty를 구현함

In [89]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate.split())
    ref_len = closest_ref_length(candidate, reference_list)
    
    if ca_len > ref_len:
        return 1
    
    # candidate가 비어있다면 BP=0 -> BLEU=0.0
    elif ca_len == 0:
        return 0
    else:
        return np.exp(1 - ref_len / ca_len)

* 위 함수는 BP의 수식처럼 c가 r보다 클 경우에는 1을 리턴하고, 그 외의 경우에는 $e^{1-r/c}$를 리턴함

In [91]:
# 최종적으로 bleu score 구현
def bleu_score(candidate, reference_list, weights=[0.25]*4):
    bp = brevity_penalty(candidate, reference_list) # 패널티 구하기
    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights, start=1)]
    
    # p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    
    return bp * np.exp(score)

### 2. NLTK를 사용한 BLEU 측정
---
* 파이썬 NLTK 패키지를 이용하여 BLEU 계산 가능

In [92]:
import nltk.translate.bleu_score as bleu

In [93]:
candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

In [94]:
# 구현한 함수
bleu_score(candidate, references)

0.5045666840058485

In [95]:
# NLTK 패키지
reference_list = [ref.split() for ref in references]

bleu.sentence_bleu(reference_list, candidate.split())

0.5045666840058485